# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {'use_vol': [0], 'window_low_bound': [1], 
                            'window_high_bound': [6],
                            'first_candle': [0.5, 0.6, 0.7, 0.8, 0.9], 
                            'second_candle': [0.5, 0.6, 0.7, 0.8, 0.9], 
                            'third_candle': [0.5]},
                'LinearReg': {'timeperiod': [8], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 25


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [18:39<00:00, 44.80s/it]


In [2]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
24,Pattern_LinearReg,0,1,6,0.9,0.9,0.5,8,0,352,1.591387,2.808750,208.168400,988.680000,1.0950,1.2768,1.3392,1.3931,1.4495,1.5462,1.5334,1.5940,1.5768,1.6117,1.6331,1.6705,1.6921,1.7104,1.6753,1.6639,1.6708,1.6577,1.6972,1.6957,1.6803,1.7297,1.7834,1.8175,0.91,1.43,1.62,1.84,1.94,2.17,2.11,2.49,2.56,2.64,2.86,2.75,2.90,3.05,3.43,3.35,3.30,3.51,3.52,3.40,3.65,3.84,3.93,4.21
1,Pattern_LinearReg,0,1,6,0.5,0.6,0.5,8,0,2639,1.444104,3.141250,1171.990896,8289.758750,1.0899,1.1730,1.2708,1.3319,1.3592,1.4150,1.4439,1.4498,1.4446,1.4142,1.4261,1.4527,1.4627,1.4828,1.4781,1.4723,1.4871,1.4930,1.5245,1.5546,1.5652,1.5918,1.6236,1.6517,0.94,1.43,1.77,1.94,2.17,2.36,2.50,2.68,2.81,3.01,3.16,3.23,3.28,3.37,3.56,3.74,3.86,3.91,4.02,4.15,4.21,4.28,4.45,4.56
14,Pattern_LinearReg,0,1,6,0.7,0.9,0.5,8,0,704,1.433062,2.954167,304.876000,2079.733333,1.0185,1.1758,1.2413,1.3125,1.3369,1.4397,1.4698,1.4833,1.4363,1.4364,1.4676,1.4944,1.5096,1.5319,1.4923,1.4457,1.4539,1.4563,1.4864,1.5015,1.4996,1.5306,1.5714,1.6018,0.90,1.58,1.74,1.92,2.04,2.33,2.49,2.66,2.87,2.94,3.05,2.96,3.01,3.21,3.45,3.46,3.44,3.55,3.65,3.72,3.76,3.89,3.99,4.29
2,Pattern_LinearReg,0,1,6,0.5,0.7,0.5,8,0,2221,1.432017,3.098750,959.509017,6882.323750,1.0489,1.1343,1.2652,1.3331,1.3496,1.4065,1.4353,1.4443,1.4312,1.4107,1.4148,1.4471,1.4562,1.4746,1.4634,1.4602,1.4727,1.4791,1.5126,1.5434,1.5519,1.5797,1.6109,1.6427,0.91,1.40,1.78,1.94,2.13,2.32,2.45,2.71,2.82,2.99,3.09,3.19,3.22,3.32,3.49,3.66,3.75,3.80,3.95,4.08,4.16,4.28,4.39,4.54
0,Pattern_LinearReg,0,1,6,0.5,0.5,0.5,8,0,2968,1.430771,3.124583,1278.527833,9273.763333,1.0651,1.1489,1.2601,1.3209,1.3454,1.3946,1.4338,1.4406,1.4404,1.4076,1.4159,1.4427,1.4437,1.4617,1.4594,1.4525,1.4681,1.4788,1.5094,1.5424,1.5552,1.5860,1.6187,1.6466,0.94,1.44,1.77,1.93,2.17,2.32,2.51,2.68,2.80,2.99,3.15,3.22,3.26,3.33,3.55,3.66,3.81,3.88,4.00,4.13,4.17,4.27,4.46,4.55
3,Pattern_LinearReg,0,1,6,0.5,0.8,0.5,8,0,1667,1.421654,3.019167,702.897496,5032.950833,1.0360,1.1213,1.2264,1.2990,1.3159,1.3813,1.4234,1.4170,1.4021,1.3798,1.3973,1.4259,1.4507,1.4760,1.4587,1.4493,1.4625,1.4738,1.5154,1.5504,1.5597,1.5940,1.6327,1.6711,0.90,1.39,1.73,1.91,2.08,2.24,2.39,2.63,2.79,2.92,3.06,3.05,3.11,3.19,3.41,3.53,3.61,3.68,3.87,3.93,4.03,4.19,4.33,4.49
4,Pattern_LinearReg,0,1,6,0.5,0.9,0.5,8,0,1002,1.421225,2.950000,422.067450,2955.900000,1.0542,1.1831,1.2670,1.3409,1.3660,1.4509,1.4687,1.4373,1.3865,1.3817,1.4058,1.4382,1.4613,1.4893,1.4434,1.4029,1.4245,1.4343,1.4734,1.5108,1.5177,1.5549,1.5872,1.6294,0.91,1.49,1.74,1.94,2.10,2.32,2.44,2.59,2.78,2.87,2.92,2.94,2.96,3.12,3.36,3.43,3.50,3.55,3.65,3.80,3.92,4.03,4.10,4.34
7,Pattern_LinearReg,0,1,6,0.6,0.7,0.5,8,0,1919,1.416288,3.045833,798.855713,5844.954167,1.0499,1.1626,1.2799,1.3314,1.3376,1.3980,1.4369,1.4397,1.4312,1.4036,1.4068,1.4386,1.4455,1.4642,1.4494,1.4416,1.4438,1.4478,1.4745,1.4968,1.5024,1.5317,1.5737,1.6033,0.94,1.47,1.80,1.95,2.13,2.32,2.47,2.74,2.84,2.98,3.08,3.17,3.21,3.30,3.46,3.59,3.64,3.66,3.77,3.88,3.95,4.11,4.26,4.38
6,Pattern_LinearReg,0,1,6,0.6,0.6,0.5,8,0,2300,1.414246,3.08833

# Check local statistics

In [2]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
71,Pattern_LinearReg,-1,1,6,0.8,0.7,0.5,8,0,695,1.715071,0.520000,496.974229,361.400000,1.0074,1.1020,1.0940,1.1651,1.8522,1.8394,1.8354,1.7891,1.7376,1.7629,1.8117,1.8146,1.7924,1.7851,1.7947,1.8131,1.8444,1.8807,1.8626,1.8683,1.9016,1.9321,1.9354,1.9399,0.32,0.31,0.33,0.34,0.49,0.43,0.42,0.38,0.37,0.45,0.47,0.52,0.46,0.49,0.51,0.63,0.65,0.70,0.70,0.67,0.76,0.77,0.72,0.59
67,Pattern_LinearReg,-1,1,6,0.8,0.6,0.5,8,0,895,1.675829,0.567083,604.867104,507.539583,1.0086,1.0568,1.0507,1.1042,1.6602,1.6673,1.6852,1.6874,1.6723,1.7105,1.7613,1.7658,1.7611,1.7788,1.8053,1.8197,1.8473,1.8817,1.8656,1.8768,1.8999,1.9427,1.9559,1.9548,0.33,0.32,0.32,0.35,0.44,0.40,0.42,0.41,0.41,0.50,0.47,0.55,0.49,0.56,0.62,0.75,0.73,0.76,0.77,0.76,0.86,0.88,0.78,0.73
63,Pattern_LinearReg,-1,1,6,0.8,0.5,0.5,8,0,1026,1.634500,0.573750,650.997000,588.667500,1.0313,1.0635,1.0633,1.1199,1.6290,1.6354,1.6569,1.6607,1.6369,1.6736,1.7152,1.7124,1.7109,1.7291,1.7530,1.7659,1.7883,1.8112,1.7933,1.8082,1.8337,1.8669,1.8811,1.8883,0.34,0.33,0.34,0.37,0.46,0.43,0.43,0.46,0.42,0.49,0.48,0.56,0.51,0.56,0.63,0.75,0.72,0.76,0.75,0.76,0.87,0.85,0.78,0.72
51,Pattern_LinearReg,-1,1,6,0.7,0.7,0.5,8,0,914,1.632650,0.514583,578.242100,470.329167,0.9698,1.0469,1.0562,1.1248,1.6911,1.7113,1.7200,1.6929,1.6479,1.6626,1.7029,1.7109,1.7179,1.6951,1.7108,1.7340,1.7604,1.8002,1.7907,1.7941,1.8302,1.8682,1.8698,1.8749,0.34,0.32,0.32,0.34,0.45,0.45,0.46,0.39,0.41,0.46,0.46,0.54,0.49,0.53,0.51,0.62,0.57,0.65,0.66,0.64,0.72,0.68,0.72,0.62
47,Pattern_LinearReg,-1,1,6,0.7,0.6,0.5,8,0,1176,1.584392,0.539167,687.244600,634.060000,0.9901,1.0255,1.0249,1.0720,1.5252,1.5504,1.5737,1.5872,1.5718,1.5996,1.6402,1.6516,1.6692,1.6700,1.7040,1.7214,1.7409,1.7748,1.7668,1.7779,1.8059,1.8527,1.8640,1.8656,0.35,0.32,0.31,0.34,0.41,0.43,0.43,0.40,0.45,0.46,0.46,0.51,0.49,0.55,0.60,0.67,0.66,0.68,0.69,0.69,0.77,0.78,0.77,0.72
43,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,8,0,1357,1.572608,0.562500,777.029508,763.312500,1.0107,1.0404,1.0465,1.1024,1.5180,1.5438,1.5723,1.5847,1.5647,1.5896,1.6283,1.6394,1.6572,1.6575,1.6895,1.7049,1.7247,1.7499,1.7348,1.7472,1.7751,1.8117,1.8210,1.8283,0.36,0.33,0.34,0.38,0.45,0.45,0.46,0.45,0.47,0.49,0.47,0.54,0.51,0.58,0.62,0.70,0.67,0.71,0.70,0.71,0.80,0.78,0.78,0.75
31,Pattern_LinearReg,-1,1,6,0.6,0.7,0.5,8,0,1180,1.566383,0.520000,668.332333,613.600000,0.9510,0.9895,1.0199,1.1030,1.5656,1.5732,1.6006,1.6157,1.5738,1.5883,1.6316,1.6449,1.6506,1.6249,1.6493,1.6677,1.6995,1.7389,1.7429,1.7395,1.7749,1.8053,1.8167,1.8259,0.35,0.33,0.32,0.33,0.45,0.45,0.47,0.44,0.45,0.45,0.44,0.47,0.47,0.51,0.51,0.64,0.65,0.67,0.64,0.64,0.70,0.68,0.72,0.70
11,Pattern_LinearReg,-1,1,6,0.5,0.7,0.5,8,0,1377,1.563804,0.516250,776.358337,710.876250,0.9451,0.9839,1.0166,1.0888,1.4895,1.5277,1.5636,1.5828,1.5696,1.5840,1.6219,1.6434,1.6477,1.6316,1.6600,1.6903,1.7174,1.7549,1.7568,1.7518,1.7931,1.8213,1.8403,1.8492,0.34,0.33,0.32,0.34,0.45,0.43,0.42,0.41,0.45,0.44,0.41,0.47,0.47,0.53,0.53,0.63,0.65,0.67,0.67,0.67,0.70,0.69,0.72,0.65
171,Pattern_LinearReg,0,1,6,0.8,0.7,0.5,8,0,1149,1.534212,0

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [11]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {'window_low_bound': [1], 'window_high_bound': [6],
                            'first_candle': [0.5], 'second_candle': [0.5]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)